In [95]:
from collections import defaultdict
from time import sleep
import os
import re
import requests
import random
from tika import parser

In [96]:
def get_draft_urls(sejm_url = ""):
    all_statutes = requests.get(sejm_url)
    pattern = r"Sejm\d.nsf/PrzebiegProc.xsp\?nr=\d+"
    urls = re.findall(pattern,all_statutes.text)
    print("Extracted all statute URLs")
    return urls

In [ ]:
urls = get_draft_urls("http://www.sejm.gov.pl/sejm8.nsf/agent.xsp?symbol=USTAWYALL&NrKadencji=8&NrPosiedzenia=86")

In [3]:
def filter_draft_urls(draft_urls):
    filtered_urls = {"poselski": [], "rzadowy": [], "obywatelski": [], "prezydencki": [], "senacki": [], "komisyjny": []}
    print("Filtering {} URLs...".format(len(draft_urls)))
    i = 0
    for url in draft_urls:
        url = "http://www.sejm.gov.pl/" + url
        sleep_time = 1 + 2 * random.random()
        sleep(sleep_time)
        i += 1
        if i % 10 == 0:
            print("#")
        if i % 100 == 0: 
            print("Done with {} of {} URLs...".format(i, len(draft_urls)))
        page = requests.get(url).text
        if page and page.find("Poselski projekt ustawy") > -1:
            filtered_urls["poselski"].append(url)
        elif page and page.find("Rządowy projekt ustawy") > -1:
            filtered_urls["rzadowy"].append(url)
        elif page and page.find("Obywatelski projekt ustawy") > -1:
            filtered_urls["obywatelski"].append(url)
        elif page and page.find("Przedstawiony przez Prezydenta") > -1:
            filtered_urls["prezydencki"].append(url)
        elif page and page.find("Senacki projekt ustawy") > -1:
            filtered_urls["senacki"].append(url)
        elif page and page.find("Komisyjny projekt ustawy") > -1:
            filtered_urls["komisyjny"].append(url)    
        
    return filtered_urls

In [4]:
categories = filter_draft_urls(urls)

Filtering 923 URLs...
#
#
#
#
#
#
#
#
#
#
Done with 100 of 923 URLs...
#
#
#
#
#
#
#
#
#
#
Done with 200 of 923 URLs...
#
#
#
#
#
#
#
#
#
#
Done with 300 of 923 URLs...
#
#
#
#
#
#
#
#
#
#
Done with 400 of 923 URLs...
#
#
#
#
#
#
#
#
#
#
Done with 500 of 923 URLs...
#
#
#
#
#
#
#
#
#
#
Done with 600 of 923 URLs...
#
#
#
#
#
#
#
#
#
#
Done with 700 of 923 URLs...
#
#
#
#
#
#
#
#
#
#
Done with 800 of 923 URLs...
#
#
#
#
#
#
#
#
#
#
Done with 900 of 923 URLs...
#
#


In [10]:
print("Poselskie: {}".format(len(categories["poselski"])))
print("Rzadowe: {}".format(len(categories["rzadowy"])))
print("Prezydenckie: {}".format(len(categories["prezydencki"])))
print("Komisyjne: {}".format(len(categories["komisyjny"])))
print("Senackie: {}".format(len(categories["senacki"])))

Poselskie: 180
Rzadowe: 657
Prezydenckie: 26
Komisyjne: 25
Senackie: 33


In [13]:
def get_pdf_url(draft_url, term = "8"):
    draft_no = draft_url[draft_url.find("=")+1:]
    source_url = "http://www.sejm.gov.pl/Sejm{}.nsf/druk.xsp?nr={}".format(term, draft_no)
    source = requests.get(source_url)
    filename = "/{}.pdf".format(draft_no)
    resource_pattern = r"http.+" + filename
    resource_url = re.findall(resource_pattern, source.text)
    return resource_url[0]

In [15]:
urls_poselskie = []
for url in categories["poselski"]:
    sleep(random.random())
    try:
        urls_poselskie.append(get_pdf_url(url))
    except:
        print("Failed to append: {}".format(url))

Failed to append: http://www.sejm.gov.pl/Sejm8.nsf/PrzebiegProc.xsp?nr=106
Failed to append: http://www.sejm.gov.pl/Sejm8.nsf/PrzebiegProc.xsp?nr=10
Failed to append: http://www.sejm.gov.pl/Sejm8.nsf/PrzebiegProc.xsp?nr=12


In [18]:
urls_poselskie = urls_poselskie + ["http://orka.sejm.gov.pl/Druki8ka.nsf/0/2BDD3075AA4884F1C1257F1B003546CE/%24File/druk%20nr%20106.pdf",
                                  "http://orka.sejm.gov.pl/Druki8ka.nsf/0/683F6A51545668D3C1257EFF005D501A/%24File/druk%20nr%2010.pdf",
                                  "http://orka.sejm.gov.pl/Druki8ka.nsf/0/0527EC5BED627495C1257F0000473007/%24File/druk%20nr%2012.pdf"]

In [56]:
def download_pdf(url):
    local_filename = "dst/" + url.split('/')[-1]
    r = requests.get(url)
    sleep(1)
    f = open(local_filename, 'wb')
    for chunk in r.iter_content(chunk_size=512 * 1024): 
        if chunk: # filter out keep-alive new chunks
            f.write(chunk)
    f.close()
    return local_filename

In [ ]:
for url in url_poselskie:
    download_pdf(url)

In [21]:
### Change this to your local path.

os.chdir("D:/ustawy")

In [98]:
### We need to parse the PDFs locally. Tika seems to be the most accurate available tool.

def extract_sponsors(draft_pdf):
    _to_remove = (";", "\n", "\t", ".", ",")
    draft = parser.from_file(draft_pdf)['content']
    start = draft.find("(-)")
    end = draft[start:].find(".") + start
    temp_sponsors = " ".join(draft[start:end].split())
    for c in _to_remove:
        temp_sponsors = temp_sponsors.replace(c, "")
    temp_sponsors = temp_sponsors.replace("(- )","(-)")
    temp_sponsors = temp_sponsors.replace("(-)", ";")[1:].split(";")
    sponsors = [s.strip() for s in temp_sponsors]
    return sponsors

In [60]:
def calculate_stats(dir = ""):
    i = 0
    bad_files = []
    sponsor_stats = defaultdict(int)
    for filename in os.listdir(os.curdir):
        i += 1
        try:    
            if filename.endswith(".pdf"): 
                sponsors = extract_sponsors(filename)
                for sponsor in sponsors:
                    sponsor_stats[sponsor] += 1
                print("{} OK.".format(filename), end = ' ')
            else:
                continue
        except Exception as e:
            print("\nBad file: {}".format(filename))
            print(e.args)
            bad_files.append(filename)
    print("{} files checked. {} bad files".format(i, len(bad_files)))
    return sponsor_stats, bad_files

In [76]:
good_stats, bad_stats = calculate_stats()


Bad file: 10.pdf
("'NoneType' object has no attribute 'find'",)
103.pdf OK. 1032.pdf OK. 104.pdf OK. 1044.pdf OK. 1047.pdf OK. 105.pdf OK. 1058.pdf OK. 1059.pdf OK. 
Bad file: 106.pdf
("'NoneType' object has no attribute 'find'",)
107.pdf OK. 1126.pdf OK. 1127.pdf OK. 1143.pdf OK. 1144.pdf OK. 115.pdf OK. 119.pdf OK. 
Bad file: 12.pdf
("'NoneType' object has no attribute 'find'",)
120.pdf OK. 121.pdf OK. 1213.pdf OK. 122.pdf OK. 1256.pdf OK. 1257.pdf OK. 1265.pdf OK. 1324.pdf OK. 1334.pdf OK. 134.pdf OK. 135.pdf OK. 136.pdf OK. 1364.pdf OK. 1412.pdf OK. 1456.pdf OK. 1491.pdf OK. 1495.pdf OK. 150.pdf OK. 1537.pdf OK. 154.pdf OK. 158.pdf OK. 162.pdf OK. 163.pdf OK. 1654.pdf OK. 1661.pdf OK. 1689.pdf OK. 171.pdf OK. 1727.pdf OK. 1733.pdf OK. 1750.pdf OK. 1849.pdf OK. 1904.pdf OK. 1919.pdf OK. 1970.pdf OK. 1977.pdf OK. 1988.pdf OK. 2001.pdf OK. 2008.pdf OK. 2033.pdf OK. 2084.pdf OK. 211.pdf OK. 2115.pdf OK. 212.pdf OK. 2131.pdf OK. 2160.pdf OK. 2183.pdf OK. 219.pdf OK. 22.pdf OK. 220.pdf 

In [79]:
### List files for manual insertion.

print(bad_stats)

['10.pdf', '106.pdf', '12.pdf', '812.pdf']


In [80]:
### Shallow copy for further work

stats = good_stats.copy()

In [81]:
stats["Andrzej Gawron"] += stats["Andrzej januszmarkiewicz Stamp Kryj"]
del stats["Andrzej januszmarkiewicz Stamp Kryj"]

In [82]:
stats["Anna Schmidt-Rodziewicz"] += stats["Anna Schmidt- Rodziewicz"]
del stats["Anna Schmidt- Rodziewicz"]
stats["Beata Mateusiak-Pielucha"] += stats["Beata Mateusiak- Pielucha"]
del stats["Beata Mateusiak- Pielucha"]
stats["Beata Małecka-Libera"] += stats["Beata Małecka- Libera"]
del stats["Beata Małecka- Libera"]
stats["Bożena Borys-Szopa"] += stats["Bożena Borys- Szopa"]
del stats["Bożena Borys- Szopa"]
stats["Dorota Arciszewska-Mielewczyk"] += stats["Dorota Arciszewska- Mielewczyk"]
del stats["Dorota Arciszewska-Mielewczyk"]
stats["Joanna Kluzik-Rostkowska"] += stats["Joanna Kluzik- Rostkowska"]
del stats["Joanna Kluzik- Rostkowska"]
stats["Józefa Szczurek-Żelazko"] += stats["Józefa Szczurek- Żelazko"]
del stats["Józefa Szczurek- Żelazko"]
stats["Mirosława Stachowiak-Różecka"] += stats["Mirosława Stachowiak- Różecka"]
del stats["Mirosława Stachowiak- Różecka"]
stats["Szymon Szynkowski vel Sęk"] += stats["Szymon Szynkowski Vel Sęk"]
del stats["Szymon Szynkowski Vel Sęk"]
stats["Władysław Kosiniak-Kamysz"] += stats["Władysław Kosiniak- Kamysz"]
del stats["Władysław Kosiniak-Kamysz"]

In [83]:
## Manual data from missing files

missing = ['Barbara Bartuś', 'Barbara Bartuś', 'Włodzimierz Bernacki', 'Włodzimierz Bernacki', 'Jacek Bogucki', 
           'Jacek Bogucki', 'Lidia Burzyńska', 'Lidia Burzyńska', 'Anna Cicholska', 'Anna Cicholska', 
           'Andrzej Gawron', 'Andrzej Gawron', 'Konrad Głębocki', 'Konrad Głębocki', 'Kazimierz Gołojuch', 
           'Kazimierz Gołojuch', 'Mariusz Orion Jędrysek', 'Mariusz Orion Jędrysek', 'Piotr Kaleta', 'Piotr Kaleta', 
           'Henryk Kowalczyk', 'Henryk Kowalczyk', 'Elżbieta Kruk', 'Elżbieta Kruk', 'Joanna Lichocka', 
           'Joanna Lichocka', 'Tomasz Ławniczak', 'Tomasz Ławniczak', 'Marzena Machałek', 'Marzena Machałek', 
           'Anna Milczanowska', 'Anna Milczanowska', 'Jan Mosiński', 'Jan Mosiński', 'Kazimierz Moskal', 
           'Kazimierz Moskal', 'Jacek Osuch', 'Jacek Osuch', 'Stanisław Pięta', 'Stanisław Pięta', 
           'Stanisław Piotrowicz', 'Jolanta Szczypińska', 'Jolanta Szczypińska', 'Andrzej Szlachta', 
           'Andrzej Szlachta', 'Halina Szydełko', 'Halina Szydełko', 'Ewa Szymańska', 'Ewa Szymańska', 
           'Jan Szyszko', 'Jan Szyszko', 'Krzysztof Tchórzewski', 'Krzysztof Tchórzewski', 'Robert Telus', 
           'Robert Telus', 'Teresa Wargocka', 'Teresa Wargocka', 'Anna Zalewska', 'Anna Zalewska', 
           'Dorota Arciszewska-Mielewczyk', 'Piotr Łukasz Babiarz', 'Joanna Borowiak', 'Bożena Borys-Szopa', 
           'Anna Cicholska', 'Michał Cieślak', 'Przemysław Czarnecki', 'Anita Czerwińska', 'Leszek Dobrzyński', 
           'Ewa Filipiak', 'Jarosław Gonciarz', 'Jan Kilian', 'Ewa Kozanecka', 'Anna Krupka', 'Andrzej Kryj', 
           'Bernadeta Krynicka', 'Dariusz Kubiak', 'Krzysztof Kubów', 'Marek Kwitek', 'Bogdan Latosiński', 
           'Marzena Machałek', 'Krzysztof Maciejewski', 'Iwona Michałek', 'Aleksander Mrówczyński', 'Wojciech Murdzek', 
           'Anna Paluch', 'Jerzy Paul', 'Marek Polak', 'Mirosława Stachowiak-Różecka', 'Artur Szałabawka', 'Piotr Uruski', 
           'Krystyna Wróblewska', 'Joanna Augustynowska', 'Wojciech Bakun', 'Józef Brynkus', 'Grzegorz Furgo', 'Marta Golbik', 
           'Paweł Grabowski', 'Marek Jakubiak', 'Andrzej Kobylarz', 'Jerzy Kozłowski', 'Paweł Kukiz', 'Jakub Kulesa', 
           'Piotr Liroy-Marzec', 'Andrzej Maciejewski', 'Maciej Masłowski', 'Robert Mordak', 'Jarosław Porwich', 
           'Stefan Romecki', 'Marek Ruciński', 'Jarosław Sachajko', 'Paweł Skutecki', 'Paweł Szramka', 'Małgorzata Zwiercan']

In [84]:
for p in missing:
    stats[p] += 1

In [90]:
del stats['']

In [91]:
sorted_stats = sorted(stats.items(), key=lambda kv: kv[1], reverse=True)

In [99]:
for i, stat in enumerate(sorted_stats):
    print("{}. {}: {}".format(i+1, stat[0], stat[1]))

1. Sylwester Tułajew: 133
2. Jerzy Gosiewski: 130
3. Waldemar Andzel: 125
4. Barbara Bartuś: 121
5. Piotr Kaleta: 117
6. Joanna Borowiak: 116
7. Kazimierz Moskal: 115
8. Barbara Dziuk: 112
9. Robert Telus: 108
10. Lidia Burzyńska: 104
11. Maria Zuba: 103
12. Ewa Szymańska: 102
13. Wojciech Skurkiewicz: 101
14. Tadeusz Woźniak: 100
15. Szymon Giżyński: 99
16. Bernadeta Krynicka: 96
17. Jarosław Gonciarz: 94
18. Jan Warzecha: 93
19. Jan Mosiński: 92
20. Ewa Tomaszewska: 89
21. Jan Kilian: 86
22. Jerzy Materna: 85
23. Piotr Uściński: 85
24. Jan Krzysztof Ardanowski: 82
25. Andrzej Kryj: 82
26. Leonard Krasulski: 82
27. Robert Kołakowski: 81
28. Krzysztof Szulowski: 79
29. Ryszard Bartosik: 76
30. Witold Czarnecki: 76
31. Andrzej Matusiewicz: 75
32. Aleksander Mrówczyński: 74
33. Bogdan Rzońca: 74
34. Mieczysław Miazga: 72
35. Piotr Uruski: 72
36. Anna Milczanowska: 72
37. Krzysztof Maciejewski: 71
38. Wojciech Zubowski: 71
39. Piotr Pyzik: 70
40. Adam Ołdakowski: 69
41. Wojciech Buczak: 6